Adquiera información de los seguidores de **X** usuario en Twitter y realize las siguientes actividades:
- Clasifique la adquisicion de los datos. De todos los datos, tome el subconjunto de seguidores del usuario **X** que tengan al menos 10 seguidores
- Dependiendo de la cantidad de seguidores que tenga **Y** seguidor, asignele un color.
- Dependiendo de la cantidad de tweets que tenga **Y** seguidor, asignele un tamaño.
- Grafique los items anteriores con el mejor tipo de gráfico, justique y visualice.

Importamos el json con la información de los usuarios:

In [80]:
import math
import json
import tweepy

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import GraphRenderer, StaticLayoutProvider, Oval, Circle, LabelSet
from bokeh.palettes import Spectral11
output_notebook()

Loading BokehJS ...

In [58]:
with open('datos_usuarios.json') as f:
    data = json.load(f)

En un problema de obtención de datos, faltaron los tweets y followers del nodo raiz (Julieta2012), por ese motivo utilizamos la api para obtener esos datos, con el fin de finalizar el gráfico:

In [59]:
auth = tweepy.OAuthHandler("MwBn5s2sjQ3vrRYMXuc9BtYfw", "36Cc2pA4NN89wTXs3LgUCxSznlpa4v8PoPYw6WTLuH1BF8V5vC")
auth.set_access_token("1037434933835378694-bUg0vb50KLHiaC78YtIa2pxSdnEytU", "VHvbqFILbnDgv2xZesppbs7f1Fk2QPDBIJFB8KoYkUCUH")
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())
#api.wait_on_rate_limit=True #en caso que haya problema con el rate limit

usuario_root = api.get_user(screen_name='Julieta2012')

Definimos una función de asignacion de colores y tamaños a los usuarios

In [74]:
def get_color_tamaño_usuario (usuario):
    cantidad_de_seguidores = usuario['followers_count']
    cantidad_de_tweets = usuario['statuses_count']
    if cantidad_de_seguidores >= 0 and cantidad_de_seguidores < 250:
        color = Spectral11[0]
    elif cantidad_de_seguidores >= 250 and cantidad_de_seguidores < 500:
        color = Spectral11[1]
    elif cantidad_de_seguidores >= 500 and cantidad_de_seguidores < 750:
        color = Spectral11[2]
    elif cantidad_de_seguidores >= 750 and cantidad_de_seguidores < 1000:
        color = Spectral11[3]
    elif cantidad_de_seguidores >= 1000 and cantidad_de_seguidores < 1250:
        color = Spectral11[4]
    elif cantidad_de_seguidores >= 1250 and cantidad_de_seguidores < 1500:
        color = Spectral11[5]
    elif cantidad_de_seguidores >= 1500 and cantidad_de_seguidores < 1750:
        color = Spectral11[6]
    elif cantidad_de_seguidores >= 1750 and cantidad_de_seguidores < 2000:
        color = Spectral11[7]
    elif cantidad_de_seguidores >= 2000 and cantidad_de_seguidores < 2250:
        color = Spectral11[8]
    elif cantidad_de_seguidores >= 2500 and cantidad_de_seguidores < 2750:
        color = Spectral11[9]
    else:
        color = Spectral11[10]
    
    
    if cantidad_de_tweets >= 0 and cantidad_de_tweets < 250:
        tamaño = 15
    elif cantidad_de_tweets >= 250 and cantidad_de_tweets < 500:
        tamaño = 16
    elif cantidad_de_tweets >= 500 and cantidad_de_tweets < 750:
        tamaño = 17
    elif cantidad_de_tweets >= 750 and cantidad_de_tweets < 1000:
        tamaño = 18
    elif cantidad_de_tweets >= 1000 and cantidad_de_tweets < 1250:
        tamaño = 19
    elif cantidad_de_tweets >= 1250 and cantidad_de_tweets < 1500:
        tamaño = 20
    elif cantidad_de_tweets >= 1500 and cantidad_de_tweets < 1750:
        tamaño = 21
    elif cantidad_de_tweets >= 1750 and cantidad_de_tweets < 2000:
        tamaño = 22
    elif cantidad_de_tweets >= 2000 and cantidad_de_tweets < 2250:
        tamaño = 23
    elif cantidad_de_tweets >= 2500 and cantidad_de_tweets < 2750:
        tamaño = 24
    else:
        tamaño = 25
        
    return color, tamaño

buscamos los seguidores del usuario root y le asignamos un color de acuerdo a la cantidad de seguidores y un tamaño de acuerdo a su cantidad de tweets:

In [88]:
datos_tamaño = []
datos_colores = []
datos_usuario = []
datos_seguidores = []
datos_tweets = []

color_nodo, tamaño_nodo = get_color_tamaño_usuario(usuario_root)
datos_colores.append(color_nodo)
datos_tamaño.append(tamaño_nodo)
datos_usuario.append(usuario_root['screen_name'])
datos_seguidores.append(usuario_root['followers_count'])
datos_tweets.append(usuario_root['statuses_count'])

seguidores = data['Julieta2012']

for seguidor in seguidores[0:10]:
    color_nodo, tamaño_nodo = get_color_tamaño_usuario(seguidor)
    datos_colores.append(color_nodo)
    datos_tamaño.append(tamaño_nodo)
    datos_usuario.append(seguidor['screen_name'])
    datos_seguidores.append(seguidor['followers_count'])
    datos_tweets.append(seguidor['statuses_count'])

In [96]:
N = 11
node_indices = list(range(N))

TOOLTIPS = [
    ("index", "$index"),
    ("usuario", "@usuarios"),
    ("seguidores", "@seguidores"),
    ("tweets", "@tweets"),
]


plot= figure(title='Usuarios y Seguidores', x_range=(-2.1,2.1), y_range=(-2.1,2.1),
            toolbar_location='right',tooltips=TOOLTIPS)

graph = GraphRenderer()

graph.node_renderer.data_source.add(node_indices, 'index')
graph.node_renderer.data_source.add(datos_colores, 'color')
graph.node_renderer.data_source.add(datos_tamaño, 'node_size')
graph.node_renderer.data_source.add(datos_usuario, 'usuarios')
graph.node_renderer.data_source.add(datos_seguidores, 'seguidores')
graph.node_renderer.data_source.add(datos_tweets, 'tweets')

graph.node_renderer.glyph = Circle(size='node_size', fill_color='color')

graph.edge_renderer.data_source.data = dict(
    start=[0]*N,
    end=node_indices)

### start of layout code
circ = [i*2*math.pi/N for i in node_indices]
x = [math.cos(i) for i in circ]
y = [math.sin(i) for i in circ]

graph_layout = dict(zip(node_indices, zip(x, y)))
graph.layout_provider = StaticLayoutProvider(graph_layout=graph_layout)

plot.renderers.append(graph)

show(plot)